In [1]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO
from google.datalab import Context
import random, string
import numpy as np
import matplotlib.pyplot as plt

In [2]:
shared_bucket = storage.Bucket('line_capstone')
for obj in shared_bucket.objects():
  if obj.key.find('/') < 0:
    print(obj.key) # Bucket 내에 포함된 데이터 셋 확인 

clicks_test.csv
clicks_train.csv
documents_categories.csv
documents_entities.csv
documents_meta.csv
documents_topics.csv
events.csv
page_views_sample.csv
promoted_content.csv
sample_submission.csv


In [3]:
my_bucket = storage.Bucket('line_capstone')

docu_meta = my_bucket.object('documents_meta.csv')
uri5 = docu_meta.uri
%gcs read --object $uri5 --variable c_docu_meta
docu_meta = pd.read_csv(BytesIO(c_docu_meta))

pro_cont = my_bucket.object('promoted_content.csv')
uri9 = pro_cont.uri
%gcs read --object $uri9 --variable c_pro_cont
promoted = pd.read_csv(BytesIO(c_pro_cont))

events = my_bucket.object('events.csv')
uri7 = events.uri
%gcs read --object $uri7 --variable c_events
events = pd.read_csv(BytesIO(c_events))

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# promoted와 docu_meta를 document_id를 key로 해서 merge
merged = pd.merge(events, docu_meta, how='inner', on='document_id')

In [5]:
# docu_meta에 NaN 값이 이미 많다.
pd.isnull(docu_meta).sum()

document_id           0
source_id          2518
publisher_id      64024
publish_time    1011118
dtype: int64

In [6]:
# evenets의 NaN 값.
pd.isnull(events).sum()

display_id        0
uuid              0
document_id       0
timestamp         0
platform          0
geo_location    340
dtype: int64

In [7]:
# merge했을 때의 NaN 값.
pd.isnull(merged).sum()

display_id            0
uuid                  0
document_id           0
timestamp             0
platform              0
geo_location        340
source_id          1322
publisher_id       1323
publish_time    3566273
dtype: int64

In [8]:
# 필요한 변수들만 할당
new_merged = merged.loc[:,['document_id','timestamp','publish_time']]

In [9]:
# from datetime import datetime
# def convert_odd_timestamp(timestamp_ms_relative):
#     TIMESTAMP_DELTA=1465876799998
#     return datetime.fromtimestamp((int(timestamp_ms_relative)+TIMESTAMP_DELTA)//1000)

In [10]:
# timestamp를 datetime으로 변환시키는 함수 정의
import datetime
def my_date(raw_date):
  added_date = raw_date + 1465876799998 # timestamp에 캐글 공식 설명에 있던 숫자를 더해줌.
  revised_date = pd.to_datetime((added_date // 1000), unit='s') # '밀리초'를 '초'로 바꾸고, string이기 때문에 unit='s' 옵션 설정 후 datetime으로 변환.
  return revised_date

In [11]:
# 위에서 정의한 함수를 사용하여 변환 후 새로운 변수에 할당.
new_merged['new_timestamp'] = my_date(new_merged['timestamp'])

In [12]:
# new_merged.sort_values(new_merged['new_timestamp'], ascending=False)
new_merged.isnull().sum()

document_id            0
timestamp              0
publish_time     3566273
new_timestamp          0
dtype: int64

In [13]:
# new_merged['new_publish_time'] = pd.to_datetime(new_merged['publish_time'])

In [14]:
new_merged['publish_time'].tail()

23120121    2016-06-28 22:00:00
23120122    2014-09-29 00:00:00
23120123    2016-06-28 23:00:00
23120124    2015-09-08 18:00:00
23120125    2016-06-28 17:00:00
Name: publish_time, dtype: object

In [18]:
span = pd.period_range('2016-06-14', '2016-06-28', freq='D')

In [19]:
span

PeriodIndex(['2016-06-14', '2016-06-15', '2016-06-16', '2016-06-17',
             '2016-06-18', '2016-06-19', '2016-06-20', '2016-06-21',
             '2016-06-22', '2016-06-23', '2016-06-24', '2016-06-25',
             '2016-06-26', '2016-06-27', '2016-06-28'],
            dtype='period[D]', freq='D')

In [17]:
# utc=True로 바꿔보고 다시 돌려보기
# 14일 04:00부터 시작하는 게 맞음?



In [ ]:
# 아래의 코드들은 아직 실행시키지 않음.

In [27]:
new_merged.head()

,document_id,timestamp,publish_time
0,379743,61.0,NaN
1,379743,74476960.0,NaN
2,379743,78297892.0,NaN
3,379743,148985378.0,NaN
4,379743,177159039.0,NaN


In [20]:
# publish_time이 이상한 게 있다.
# 1412년에 발행된 글들이 2870개나 된다.
new_merged[new_merged['publish_time'] == '1412-01-24 00:00:00'].shape

(2870, 4)

In [23]:
# 1970년 1월 1일 이전에 발행된 글들이 3,229개나 된다.
new_merged[new_merged['publish_time'] < '1970-01-01 00:00:00'].shape

(3229, 4)

In [254]:
# publish_time에 있는 비정상적인 값들을 수정.
# 예를 들어, 1970년 이전 발행글들은 1970년으로 수정하고 2016년 6월 18일 이후는 18일로 수정.
# new_merged['new_publish_time'] = new_merged['publish_time'].replace(('1412-01-24 00:00:00'), '1970-01-01 00:00:00')
# new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('0002-11-27 19:00:00'), '1970-01-01 00:00:00')
# new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('0017-06-30 19:00:00'), '1970-01-01 00:00:00')
new_merged.ix[new_merged['new_publish_time'] < '1970-01-01 00:00:00'] = '1970-01-01 00:00:00'
new_merged['new_publish_time'] = new_merged['new_publish_time'].replace(('3012-05-03 00:00:00'), '2016-06-28 00:00:00')

In [263]:
# 데이터 타입 변환(위에서 해결했으니 나중에 지우기)
# new_merged['new_timestamp'] = pd.to_datetime(new_merged['new_timestamp'], format='%Y-%m-%d %H:%M:%S', utc=True)
new_merged['new_publish_time'] = pd.to_datetime(new_merged['new_publish_time'], format='%Y-%m-%d %H:%M:%S')